In [1]:
%pip install git+https://github.com/vanderschaarlab/temporai.git
import pandas as pd
import numpy as np
from random import randrange, uniform

from google.colab import drive
drive.mount('/content/drive')

  Cloning https://github.com/vanderschaarlab/temporai.git to /tmp/pip-req-build-23wk_2pd
  Running command git clone --filter=blob:none --quiet https://github.com/vanderschaarlab/temporai.git /tmp/pip-req-build-23wk_2pd
  Resolved https://github.com/vanderschaarlab/temporai.git to commit 7efd3e2330918337a819bcb810235699dfb45528
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Mounted at /content/drive


In [2]:
colunas = ['eventdeath', 'survival', 'timerecurrence', 'chemo', 'hormonal', 'amputation', 'histtype', 'diam', 'posnodes', 'grade', 'angioinv', 'lymphinfil' ]
df = pd.read_csv('/content/drive/MyDrive/Data/nki_cleaned.csv', usecols=colunas)

index_count = []
random_num = []

for i in range(1, len(df.index)+1):
  index_count.append(i)
  random_num.append(randrange(1, 20))

df['sample_idx'] = index_count
df['time_idx'] = random_num

In [8]:
time_series_df = df.copy()
time_series_df.set_index(keys=["sample_idx", "time_idx"], drop=True, inplace=True)

In [9]:
time_series_df

,,eventdeath,survival,timerecurrence,chemo,hormonal,amputation,histtype,diam,posnodes,grade,angioinv,lymphinfil
sample_idx,time_idx,,,,,,,,,,,,
1,11,False,14.817248,14.817248,False,False,True,1,25,0,2,3,1
2,5,False,14.261465,14.261465,False,False,False,1,20,0,3,3,1
3,10,False,6.644764,6.644764,False,False,False,1,15,0,2,1,1
4,8,False,7.748118,7.748118,False,True,False,1,15,1,2,3,1
5,1,False,6.436687,6.318960,False,False,True,1,15,0,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
268,19,True,2.608300,1.982204,True,False,True,1,30,0,3,1,3
269,17,True,5.504100,3.028063,False,False,True,1,30,0,2,1,1
270,5,True,2.619200,2.149213,False,False,True,1,27,0,3,1,1


In [4]:
event_df = df.copy()
event_df.set_index(keys=["sample_idx"], drop=True, inplace=True)

event_df['event'] = event_df[['time_idx', 'eventdeath']].apply(tuple, axis=1)
event_df.drop(event_df.iloc[:, :13], inplace=True, axis=1)

In [10]:
event_df

,event
sample_idx,
1,"(11, False)"
2,"(5, False)"
3,"(10, False)"
4,"(8, False)"
5,"(1, False)"
...,...
268,"(19, True)"
269,"(17, True)"
270,"(5, True)"


In [5]:
from tempor.data.dataset import TimeToEventAnalysisDataset

data = TimeToEventAnalysisDataset(
    time_series=time_series_df,
    targets=event_df,
)

In [6]:
from tempor import plugin_loader

model = plugin_loader.get("time_to_event.dynamic_deephit", n_iter=50)
model.fit(data)
model.predict(data, horizons=[0.25, 0.50, 0.75])

RuntimeError: NaNs detected in the total loss